# Kaggle - Titanic competition dataset

In [140]:
import pandas as pd
import numpy as np
import re

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('gender_submission.csv')

In [3]:
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
596,597,1,2,"Leitch, Miss. Jessie Wills",female,NaN,0,0,248727,33.0000,NaN,S
503,504,0,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.5875,NaN,S
216,217,1,3,"Honkanen, Miss. Eliina",female,27.0,0,0,STON/O2. 3101283,7.9250,NaN,S
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [5]:
def gender_transform(x):
    if x == 'female':
        return 1
    else: 
        return 0
    
train['Gender_Bi'] = train['Sex'].map(gender_transform)
test['Gender_Bi'] = test['Sex'].map(gender_transform)

In [7]:
variables = ['Gender_Bi', 'Fare']

X = train[variables]
y = train['Survived']

In [19]:
X = X.fillna(-1)

In [40]:
model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
model.fit(X, y)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [30]:
X_prev = test[variables]
X_prev = X_prev.fillna(-1)

In [32]:
p = model.predict(X_prev)
p

array([0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [36]:
### First Submission 

sub = pd.Series(p, index=test['PassengerId'], name='Survived')

In [37]:
sub.to_csv('first_sub.csv', header=True)

### Finding Patterns in the name

In [145]:
# Search for context aspects 

titles = ['Miss', 'Mr', 'Master', 'Dr', 'Mrs', 'Rev', 'Countess']


def find_title_in_name(x):
    for title in titles: 
        #print('Finding:', strategy)
        if re.search(title, x): 
            #print('Found:', strategy)
            return title


train['Title'] = train['Name'].map(find_title_in_name)
test['Title'] = test['Name'].map(find_title_in_name)

# Title Categorical Variable Transformation

def title_adjust(y):
    if y == 'Miss':
        return 1
    if y == 'Mr':
        return 2
    if y == 'Master': 
        return 3
    if y == 'Dr': 
        return 4
    if y == 'Mrs': 
        return 5 
    if y == 'Rev':
        return 6
    if y == 'Countess':
        return 7
    
train['Title'] = train['Title'].map(title_adjust)
test['Title'] = test['Title'].map(title_adjust)

train = train.fillna(0)

train['Title'] = train['Title'].astype(np.int64)

In [146]:
train.sample(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender_Bi,Title
446,447,1,2,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,0,S,1,1
627,628,1,1,"Longley, Miss. Gretchen Fiske",female,21.0,0,0,13502,77.9583,D9,S,1,1
570,571,1,2,"Harris, Mr. George",male,62.0,0,0,S.W./PP 752,10.5000,0,S,0,2
90,91,0,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,0,S,0,2
151,152,1,1,"Pears, Mrs. Thomas (Edith Wearne)",female,22.0,1,0,113776,66.6000,C2,S,1,2
411,412,0,3,"Hart, Mr. Henry",male,0.0,0,0,394140,6.8583,0,Q,0,2
87,88,0,3,"Slocovski, Mr. Selman Francis",male,0.0,0,0,SOTON/OQ 392086,8.0500,0,S,0,2
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,0,C,1,1


## Validation

In [46]:
# Create train and valid variables

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5)

In [47]:

model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
model.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [48]:
p2 = model.predict(X_valid)
p2

array([0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,

In [49]:
# Accuracy

np.mean(y_valid == p2)

0.7914798206278026

### Cross-validation

In [51]:
from sklearn.model_selection import KFold

In [72]:
results = []

for rep in range(10):

    kf = KFold(10, shuffle=True, random_state = 0)

    for train_lines, valid_lines in kf.split(X):
        X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
        y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]

        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p3 = model.predict(X_valid)

        acc = np.mean(y_valid == p3)
        results.append(acc)

        #print(acc)


In [73]:
print('Mean:', np.mean(results))
print('Standard Deviation:', np.std(results))
print('Variance:', np.var(results))

Mean: 0.8058302122347065
Standard Deviation: 0.014321080699814881
Variance: 0.0002050933524106103


### Adding new variables

##### Passenger Class

In [74]:
train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender_Bi
645,646,1,1,"Harper, Mr. Henry Sleeper",male,48.0,1,0,PC 17572,76.7292,D33,C,0
339,340,0,1,"Blackwell, Mr. Stephen Weart",male,45.0,0,0,113784,35.5000,T,S,0
710,711,1,1,"Mayne, Mlle. Berthe Antonine (""Mrs de Villiers"")",female,24.0,0,0,PC 17482,49.5042,C90,C,1
479,480,1,3,"Hirvonen, Miss. Hildur E",female,2.0,0,1,3101298,12.2875,NaN,S,1
416,417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.5000,NaN,S,1
664,665,1,3,"Lindqvist, Mr. Eino William",male,20.0,1,0,STON/O 2. 3101285,7.9250,NaN,S,0
600,601,1,2,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",female,24.0,2,1,243847,27.0000,NaN,S,1
344,345,0,2,"Fox, Mr. Stanley Hubert",male,36.0,0,0,229236,13.0000,NaN,S,0
661,662,0,3,"Badt, Mr. Mohamed",male,40.0,0,0,2623,7.2250,NaN,C,0
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S,0


In [75]:
variables = ['Gender_Bi', 'Fare', 'Pclass']

X = train[variables]
y = train['Survived']

In [76]:
results = []

for rep in range(10):

    kf = KFold(10, shuffle=True, random_state = 0)

    for train_lines, valid_lines in kf.split(X):
        X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
        y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]

        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p3 = model.predict(X_valid)

        acc = np.mean(y_valid == p3)
        results.append(acc)

# Results measures         
        
print('Mean:', np.mean(results))
print('Standard Deviation:', np.std(results))
print('Variance:', np.var(results))

Mean: 0.8193133583021222
Standard Deviation: 0.016888395892075966
Variance: 0.0002852179158074884


##### Age

In [77]:
variables = ['Gender_Bi', 'Fare', 'Pclass', 'Age']

X = train[variables]
y = train['Survived']

In [79]:
X = X.fillna(-1)

In [80]:
results = []

for rep in range(10):

    kf = KFold(10, shuffle=True, random_state = 0)

    for train_lines, valid_lines in kf.split(X):
        X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
        y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]

        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p3 = model.predict(X_valid)

        acc = np.mean(y_valid == p3)
        results.append(acc)

# Results measures         
        
print('Mean:', np.mean(results))
print('Standard Deviation:', np.std(results))
print('Variance:', np.var(results))

Mean: 0.8226716604244692
Standard Deviation: 0.01865213663097499
Variance: 0.000347902200900559


In [81]:
# Gender, Fare, Passenger Class and Age Submission

sub = pd.Series(p, index=test['PassengerId'], name='Survived')

sub.to_csv('gen_fare_pclass_age.csv', header=True)


##### Title 

In [171]:
variables = ['Title', 'Gender_Bi', 'Pclass', 'Fare', 'Age']

X = train[variables]
y = train['Survived']

In [172]:
results = []

for rep in range(10):

    kf = KFold(10, shuffle=True, random_state = 0)

    for train_lines, valid_lines in kf.split(X):
        X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
        y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]

        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p3 = model.predict(X_valid)

        acc = np.mean(y_valid == p3)
        results.append(acc)

# Results measures         
        
print('Mean:', np.mean(results))
print('Standard Deviation:', np.std(results))
print('Variance:', np.var(results))

Mean: 0.8260799001248439
Standard Deviation: 0.01467672660772338
Variance: 0.00021540630391785542


In [158]:
# Gender, Fare, Passenger Class, Age, Title Submission

sub = pd.Series(p, index=test['PassengerId'], name='Survived')

sub.to_csv('gen_fare_pclass_age_title.csv', header=True)

## Dimensionality Reduction Using Feature Selection

### Thresholding Numerical Feature Variance

In [88]:
from sklearn.feature_selection import VarianceThreshold

In [153]:
# Define features and target

variables = ['Fare', 'Pclass', 'Age', 'SibSp', 'Parch', 'Title']

X = train[variables]
y = train['Survived']

In [154]:
# Create thresholder

thresholder = VarianceThreshold(threshold=.9)

In [155]:
# Create high variance feature matrix

features_high_variance = thresholder.fit_transform(X)

In [156]:
# View high variance feature matrix

features_high_variance[0:3]

array([[ 7.25  , 22.    ,  1.    ],
       [71.2833, 38.    ,  1.    ],
       [ 7.925 , 26.    ,  0.    ]])

In [157]:
# View variances 

thresholder.fit(X).variances_

array([2.46666531e+03, 6.98230591e-01, 3.09274323e+02, 1.21467827e+00,
       6.48999031e-01, 4.43441775e-01])

##### Pclass and Parch

The result using the two features with the highest variance was worse than the result using other variables. This might be explained by the fact that the other variables, despite having lower variances, are actually extreme powerfull in explaining the target variable. 


In [119]:
# Run model with Pclass and Parch

variables = ['Pclass', 'Parch']

X = train[variables]
y = train['Survived']

results = []

for rep in range(10):

    kf = KFold(10, shuffle=True, random_state = 0)

    for train_lines, valid_lines in kf.split(X):
        X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
        y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]

        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p3 = model.predict(X_valid)

        acc = np.mean(y_valid == p3)
        results.append(acc)

# Results measures         
        
print('Mean:', np.mean(results))
print('Standard Deviation:', np.std(results))
print('Variance:', np.var(results))



Mean: 0.7092883895131087
Standard Deviation: 0.040195415434580425
Variance: 0.0016156714219585063
